To Do:
 - [X] join in NCSL
 - [X] join local opposition
 - [X] model state level ordinances
 - [ ] make resource class
 - [X] is_hybrid
 - [X] has_opposition
 - [ ] replace "Unknown" ISO counties with NULL before geocoding. It fills in garbage otherwise 

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import dbcp
import pandas as pd

/app/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
from dbcp.data_mart import counties as ct

In [5]:
engine = dbcp.helpers.get_sql_engine()

In [6]:
loc_df = ct._get_iso_location_df(engine)

In [7]:
# no multi-valued projects
assert loc_df['project_id'].nunique() == loc_df.shape[0]

In [8]:
res_df = ct._get_iso_resource_df(engine)

In [9]:
res_df.head()

,capacity_mw,project_id,resource_clean
0,500.0,0,Battery Storage
1,500.0,1,Battery Storage
2,725.0,3,Battery Storage
3,4.5,4,Onshore Wind
4,80.0,5,Battery Storage


In [10]:
proj = pd.read_csv('../data/output/project_data_mart.csv')

In [11]:
proj['project_id'].nunique()

13488

In [12]:
proj.shape

(13488, 34)

In [13]:
iso = ct._get_and_join_iso_tables()

In [14]:
iso.head()

,capacity_mw,project_id,resource_clean,queue_date,queue_status,region,county_id_fips
0,500.0,0,Battery Storage,2020-08-13,active,West (non-ISO),30111
1,500.0,1,Battery Storage,2020-07-17,active,West (non-ISO),30111
2,725.0,3,Battery Storage,2019-04-15,active,CAISO,06065
3,4.5,4,Onshore Wind,NaT,active,SPP,40047
4,80.0,5,Battery Storage,NaT,active,SPP,40153


In [15]:
iso.columns

Index(['capacity_mw', 'project_id', 'resource_clean', 'queue_date',
       'queue_status', 'region', 'county_id_fips'],
      dtype='object')

In [16]:
county_aggs = ct._agg_iso_projects_to_counties(iso)
county_aggs

breakout_fuels,battery storage,coal,gas,solar,wind,battery storage,coal,gas,solar,wind,fossil,renewable,fossil,renewable
,capacity_mw,capacity_mw,capacity_mw,capacity_mw,capacity_mw,count_plants,count_plants,count_plants,count_plants,count_plants,capacity_mw,capacity_mw,count_plants,count_plants
county_id_fips,,,,,,,,,,,,,,
01003,0.0,NaN,NaN,1200.0,NaN,2.0,NaN,NaN,4.0,NaN,NaN,1200.0,NaN,4.0
01013,NaN,NaN,NaN,80.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,80.0,NaN,1.0
01031,87.0,NaN,NaN,80.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,80.0,NaN,1.0
01033,0.0,NaN,780.0,697.0,NaN,2.0,NaN,1.0,4.0,NaN,780.0,697.0,1.0,4.0
01047,NaN,NaN,NaN,80.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,80.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.8,NaN,1.0
32009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,1.0


In [17]:
county_aggs.sum()

breakout_fuels               
battery storage  capacity_mw     157105.65
coal             capacity_mw        130.70
gas              capacity_mw      63288.00
solar            capacity_mw     431969.56
wind             capacity_mw     178182.82
battery storage  count_plants      1533.00
coal             count_plants         7.00
gas              count_plants       240.00
solar            count_plants      3388.00
wind             count_plants       714.00
fossil           capacity_mw      63288.00
renewable        capacity_mw     613931.22
fossil           count_plants       240.00
renewable        count_plants      4163.00
dtype: float64

# Existing Plants
for breakout fuel categories:

Get sum(capacity) and counts

Get sum(capacity) and counts for projects built in most recent year (2020)

In [18]:
pudl = ct._get_pudl_df(engine)

In [19]:
pudl.head()

,capacity_mw,county_id_fips,fuel_type_code_pudl,operating_date,technology_description
0,0.9,02013,oil,2000-12-01,Petroleum Liquids
1,0.9,02013,oil,2000-12-01,Petroleum Liquids
2,0.5,02013,oil,2010-12-01,Petroleum Liquids
3,0.7,02013,oil,2000-12-01,Petroleum Liquids
4,0.5,02013,wind,2011-10-01,Onshore Wind Turbine


In [20]:
pudl.shape

(23416, 5)

In [21]:
existing = ct._agg_pudl_to_counties(pudl)

In [22]:
existing.sum()

breakout_fuels               
battery storage  capacity_mw       1400.7
coal             capacity_mw     225409.3
gas              capacity_mw     532478.6
solar            capacity_mw      47064.5
wind             capacity_mw     113367.1
battery storage  count_plants       211.0
coal             count_plants       589.0
gas              count_plants      5802.0
solar            count_plants      4436.0
wind             count_plants      1355.0
fossil           capacity_mw     786977.4
renewable        capacity_mw     259181.1
fossil           count_plants      9476.0
renewable        count_plants      9618.0
dtype: float64

In [23]:
built_2020 = pudl.query('operating_date >= "2020-01-01"')

In [24]:
built_2020.shape

(860, 5)

In [25]:
ct._agg_pudl_to_counties(built_2020).sum()

breakout_fuels               
battery storage  capacity_mw       490.0
coal             capacity_mw        17.0
gas              capacity_mw      7186.8
solar            capacity_mw     10128.3
wind             capacity_mw     14049.5
battery storage  count_plants       56.0
coal             count_plants        1.0
gas              count_plants       95.0
solar            count_plants      556.0
wind             count_plants       91.0
fossil           capacity_mw      7238.8
renewable        capacity_mw     24332.4
fossil           count_plants      117.0
renewable        count_plants      653.0
dtype: float64

In [26]:
pd.Index(['prefix_' + '_'.join(levels).replace(' ', '_') for levels in county_aggs.columns])

Index(['prefix_battery_storage_capacity_mw', 'prefix_coal_capacity_mw',
       'prefix_gas_capacity_mw', 'prefix_solar_capacity_mw',
       'prefix_wind_capacity_mw', 'prefix_battery_storage_count_plants',
       'prefix_coal_count_plants', 'prefix_gas_count_plants',
       'prefix_solar_count_plants', 'prefix_wind_count_plants',
       'prefix_fossil_capacity_mw', 'prefix_renewable_capacity_mw',
       'prefix_fossil_count_plants', 'prefix_renewable_count_plants'],
      dtype='object')

In [27]:
int(pd.read_sql("select max(date_part('year', report_date)) from dbcp.mcoe;", engine).iat[0,0])

2020

In [34]:
out = ct.make_county_data_mart_table()

In [35]:
out

,state_id_fips,county_id_fips,state,county,fossil_built_in_2020_capacity_mw,fossil_built_in_2020_count_plants,fossil_existing_capacity_mw,fossil_existing_count_plants,fossil_proposed_capacity_mw,fossil_proposed_count_plants,...,wind_existing_count_plants,wind_proposed_capacity_mw,wind_proposed_count_plants,has_ordinance,ordinance_jurisdiction_name,ordinance_jurisdiction_type,ordinance,ordinance_earliest_year_mentioned,state_permitting_type,state_permitting_text
0,01,01003,AL,Baldwin County,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,Baldwin County,county,"Large wind energy conversion systems (WECS), U...",NaN,Local,According to the Wind Energy Technology Office...
1,01,01013,AL,Butler County,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Local,According to the Wind Energy Technology Office...
2,01,01031,AL,Coffee County,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Local,According to the Wind Energy Technology Office...
3,01,01033,AL,Colbert County,NaN,NaN,476.0,8.0,780.0,1.0,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Local,According to the Wind Energy Technology Office...
4,01,01047,AL,Dallas County,NaN,NaN,66.8,3.0,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Local,According to the Wind Energy Technology Office...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2282,55,55103,WI,Richland County,NaN,NaN,1.8,1.0,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Hybrid,The Wisconsin Public Service Commission has ex...
2283,55,55111,WI,Sauk County,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Hybrid,The Wisconsin Public Service Commission has ex...
2284,55,55113,WI,Sawyer County,NaN,NaN,5.6,2.0,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Hybrid,The Wisconsin Public Service Commission has ex...
2285,55,55129,WI,Washburn County,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Hybrid,The Wisconsin Public Service Commission has ex...


In [36]:
# very sparse
out.isna().mean().mean()

0.7024362475352895

In [37]:
list(out.columns)

['state_id_fips',
 'county_id_fips',
 'state',
 'county',
 'fossil_built_in_2020_capacity_mw',
 'fossil_built_in_2020_count_plants',
 'fossil_existing_capacity_mw',
 'fossil_existing_count_plants',
 'fossil_proposed_capacity_mw',
 'fossil_proposed_count_plants',
 'renewable_built_in_2020_capacity_mw',
 'renewable_built_in_2020_count_plants',
 'renewable_existing_capacity_mw',
 'renewable_existing_count_plants',
 'renewable_proposed_capacity_mw',
 'renewable_proposed_count_plants',
 'battery_storage_built_in_2020_capacity_mw',
 'battery_storage_built_in_2020_count_plants',
 'battery_storage_existing_capacity_mw',
 'battery_storage_existing_count_plants',
 'battery_storage_proposed_capacity_mw',
 'battery_storage_proposed_count_plants',
 'coal_built_in_2020_capacity_mw',
 'coal_built_in_2020_count_plants',
 'coal_existing_capacity_mw',
 'coal_existing_count_plants',
 'coal_proposed_capacity_mw',
 'coal_proposed_count_plants',
 'gas_built_in_2020_capacity_mw',
 'gas_built_in_2020_count_pl

In [38]:
!pwd

/app/notebooks


In [39]:
assert False

AssertionError: 

In [40]:
out.to_csv('../data/output/county_data_mart.csv', index=False)